In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
dfs = []
parent_file_location = os.path.dirname(os.path.abspath(os.getcwd()))
latnecy_data_path = os.path.join(parent_file_location,'results','throughput','secure')
for root, dir,files in os.walk(latnecy_data_path):
    for file in files:
        if file == 'raw_data.csv':
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            dfs.append(df)
raw_df_secure = pd.concat(dfs, ignore_index=True)
success_df_secure = raw_df_secure[raw_df_secure['return_code'] == 200]
success_df_secure['latency'] = success_df_secure['latency']*1000
success_df_secure.head()

,return_code,payload,latency,throughput
0,200,"{""signature"":""0xadfe3b1003aae2af30e50298135cac...",406.1779,60
1,200,"{""signature"":""0xb522a6e335e191b2f87f8be04d4e35...",378.5459,60
2,200,"{""signature"":""0xb1e7bdf0ed49c87de8cbd955590d1b...",371.3515,60
3,200,"{""signature"":""0xb3c721db07d84043234715db63bffd...",379.2316,60
4,200,"{""signature"":""0xb43f642e1b189bafe13cd5e98b156a...",378.1444,60


In [5]:
success_df_secure.shape

(100, 4)

In [6]:
dfs = []
parent_file_location = os.path.dirname(os.path.abspath(os.getcwd()))
latnecy_data_path = os.path.join(parent_file_location,'results','throughput','unsecure')
for root, dir,files in os.walk(latnecy_data_path):
    for file in files:
        if file == 'raw_data.csv':
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            dfs.append(df)
raw_df_unsecure = pd.concat(dfs, ignore_index=True)
raw_df_unsecure.head()
success_df_unsecure = raw_df_unsecure[raw_df_unsecure['return_code'] == 200]
success_df_unsecure['latency'] = success_df_unsecure['latency']*1000
success_df_unsecure.head()

,return_code,payload,latency,throughput
0,200,{'signature': '0xa25825bf160189b160a85980b0d02...,406.2623,60
1,200,{'signature': '0xa202d72813638e7bcf0e8d90942e5...,366.0465,60
2,200,{'signature': '0xb6ffc3c152b40b8c2ef666c3bf564...,454.8093,60
3,200,{'signature': '0x83da8041d55269c847612d82257ff...,464.4123,60
4,200,{'signature': '0x997e3abb06ad93bf5cad8510a197b...,407.8439,60


In [7]:
success_df_unsecure.shape

(100, 4)

In [9]:
grouped_results_secure = success_df_secure.groupby('throughput')
grouped_results_secure = grouped_results_secure['latency'].agg(
    sample_count=('count'),
    mean_value=('mean'),
    percentile_25=lambda x: x.quantile(0.25),
    percentile_50=lambda x: x.quantile(0.5),
    percentile_75=lambda x: x.quantile(0.75),
    percentile_95=lambda x: x.quantile(0.95)
).reset_index()

In [10]:
grouped_results_secure.head()

,throughput,sample_count,mean_value,percentile_25,percentile_50,percentile_75,percentile_95
0,60,100,382.308105,372.654975,376.82695,382.586125,432.771735


In [12]:
grouped_results_unsecure = success_df_unsecure.groupby('throughput')
grouped_results_unsecure = grouped_results_unsecure['latency'].agg(
    sample_count=('count'),
    mean_value=('mean'),
    percentile_25=lambda x: x.quantile(0.25),
    percentile_50=lambda x: x.quantile(0.5),
    percentile_75=lambda x: x.quantile(0.75),
    percentile_95=lambda x: x.quantile(0.95)
).reset_index()

In [13]:
grouped_results_unsecure.head()

,throughput,sample_count,mean_value,percentile_25,percentile_50,percentile_75,percentile_95
0,60,100,382.87355,372.149575,375.2991,381.522075,427.102455


In [15]:
merged_results = pd.merge(grouped_results_unsecure,grouped_results_secure, on='throughput', suffixes=('_unsecure','_secure'))

In [16]:
merged_results.head()

,throughput,sample_count_unasecure,mean_value_unasecure,percentile_25_unasecure,percentile_50_unasecure,percentile_75_unasecure,percentile_95_unasecure,sample_count_secure,mean_value_secure,percentile_25_secure,percentile_50_secure,percentile_75_secure,percentile_95_secure
0,60,100,382.87355,372.149575,375.2991,381.522075,427.102455,100,382.308105,372.654975,376.82695,382.586125,432.771735
